需要的信息：entryID\Taxon ID of the species\sequence   OBO中的几个term: Molecular Function (MF), Biological Process (BP), and Cellular Component (CC)
https://github.com/claradepaolis/InformationAccretion (IA文件评分)

In [27]:
import numpy as np
import pandas as pd 
import os

home_dir = os.environ.get('HOME')
directory_path = os.path.join(home_dir, 'Data', 'protein', 'cafa-5-protein-function-prediction')

for dirname, _, filenames in os.walk(directory_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/zane/Data/protein/cafa-5-protein-function-prediction/sample_submission.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/.DS_Store
/Users/zane/Data/protein/cafa-5-protein-function-prediction/IA.txt
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Test (Targets)/.DS_Store
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/.DS_Store
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/train_terms.tsv
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/go-basic.obo
/Users/zane/Data/protein/cafa-5-protein-function-prediction/Train/train_sequences.fasta


In [28]:
# !pip install Bio

In [29]:
#处理/Train/train_sequences.fasta
from Bio import SeqIO

In [30]:
sequences_dict = {}
for seq_record in SeqIO.parse(os.path.join(directory_path, 'Train', 'train_sequences.fasta'), "fasta"):
    sequences_dict[seq_record.id] = str(seq_record.seq)
sequences = pd.DataFrame(list(sequences_dict.items()), columns=['EntryID', 'Sequence'])

In [31]:
sequences_annotations = pd.read_csv(os.path.join(directory_path, 'Train', 'train_taxonomy.tsv') ,sep='\t')

In [32]:
#把两个表合并
train = pd.merge(sequences, sequences_annotations, on='EntryID')

In [33]:
#使用obonet处理/Train/go-basic.obo
# !pip install obonet

In [34]:
# !pip show pandas

In [35]:
import obonet
import pandas as pd

# 读取OBO文件并构建网络图
def read_obo(file_path):
    return obonet.read_obo(file_path)

# 提取指定命名空间的term并返回DataFrame
def extract_terms(graph, namespaces):
    terms_list = []
    for node, data in graph.nodes(data=True):
        if 'namespace' in data and data['namespace'] in namespaces:
            terms_list.append({
                'ID': node,
                'Name': data['name'],
                'Namespace': data['namespace']
            })
    return pd.DataFrame(terms_list)

# 主程序
def main(par):
    # 读取OBO文件并构建网络图
    graph = read_obo(par)
    # 提取GO ID和指定命名空间的term
    namespaces = {"molecular_function", "biological_process", "cellular_component"}
    terms_df = extract_terms(graph, namespaces)
    terms_df.columns = ['term', 'Name', 'Namespace']  # 重命名列
    return terms_df

if __name__ == '__main__':
    par = os.path.join(directory_path, 'Train', 'go-basic.obo')
    # par = "../cafa-5-protein-function-prediction/Train/go-basic.obo"
    obo_info = main(par)
    print(obo_info)


             term                                               Name  \
0      GO:0000001                          mitochondrion inheritance   
1      GO:0000002                   mitochondrial genome maintenance   
2      GO:0000003                                       reproduction   
3      GO:0000006  high-affinity zinc transmembrane transporter a...   
4      GO:0000007  low-affinity zinc ion transmembrane transporte...   
...           ...                                                ...   
43243  GO:2001313  UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...   
43244  GO:2001314  UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...   
43245  GO:2001315  UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...   
43246  GO:2001316                       kojic acid metabolic process   
43247  GO:2001317                    kojic acid biosynthetic process   

                Namespace  
0      biological_process  
1      biological_process  
2      biological_process  
3      molecular_functi

In [36]:
#读入train_terms.tsv
sequences_terms = pd.read_csv(os.path.join(directory_path, 'Train', 'train_terms.tsv') ,sep='\t')

In [37]:
obo_info2=pd.merge(obo_info, sequences_terms, on='term')

In [38]:
#这个是最终的数据表
train_total=pd.merge(train, obo_info2, on='EntryID').iloc[:,:-1]

In [39]:
train_total

,EntryID,Sequence,taxonomyID,term,Name,Namespace
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0003674,molecular_function,molecular_function
1,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0005488,binding,molecular_function
2,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0005515,protein binding,molecular_function
3,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0006139,nucleobase-containing compound metabolic process,biological_process
4,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0006259,DNA metabolic process,biological_process
...,...,...,...,...,...,...
5363858,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,10116,GO:0110165,cellular anatomical entity,cellular_component
5363859,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,10116,GO:0120025,plasma membrane bounded cell projection,cellular_component
5363860,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,10116,GO:1901363,heterocyclic compound binding,molecular_function
5363861,A0A8I6GHU0,HCISSLKLTAFFKRSFLLSPEKHLVLLRDGRTLIGFLRSIDQFANL...,10116,GO:1990124,messenger ribonucleoprotein complex,cellular_component


In [40]:
import numpy as np
import pandas as pd
import os
from Bio import SeqIO
from transformers import EsmTokenizer, EsmModel
import torch

In [41]:
from transformers import EsmTokenizer, EsmModel
import torch

# 加载预训练的ESM模型和tokenizer
model_name = "facebook/esm-1b"
tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmModel.from_pretrained(model_name)

# 输入蛋白质序列
sequence = "MKTLLVVVVVVLALAVLAAAAAAAAA"

# 将序列转换为模型输入
inputs = tokenizer(sequence, return_tensors="pt")

# 打印tokenizer的工作细节
print("Tokenized input IDs:", inputs['input_ids'])

# 获取模型输出（嵌入向量）
with torch.no_grad():
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state

# 打印嵌入向量
print("Embedding shape:", embedding.shape)
print("Embedding:", embedding)

/opt/anaconda3/envs/biotorch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ESMTokenizer'. 
The class this function is called from is 'EsmTokenizer'.
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm-1b and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight', 'esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenized input IDs: tensor([[ 0, 20, 15, 11,  4,  4,  7,  7,  7,  7,  7,  7,  4,  5,  4,  5,  7,  4,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  2]])
Embedding shape: torch.Size([1, 28, 1280])
Embedding: tensor([[[ 2.8017e-01,  3.9156e-02, -2.0031e-02,  ..., -1.7287e-02,
          -1.8021e-01, -2.0528e-01],
         [-3.2456e-02, -1.7327e-02,  3.6042e-02,  ..., -5.8190e-02,
           8.5993e-02, -4.8921e-01],
         [-6.8467e-03, -1.9018e-02, -2.2709e-02,  ..., -5.0332e-02,
           5.1199e-02, -4.7812e-01],
         ...,
         [-3.7725e-05, -5.0551e-03, -1.7264e-02,  ..., -2.8736e-02,
           6.5339e-02, -5.1349e-01],
         [-1.1798e-02,  2.4026e-03, -5.3339e-03,  ..., -3.1110e-02,
           8.3229e-02, -4.9574e-01],
         [-3.1180e-02, -8.2870e-03, -1.4255e-02,  ..., -7.7869e-03,
           9.0788e-02, -4.8007e-01]]])


In [47]:
subset_train_total = train_total.head(7)

model_name = "facebook/esm-1b"
tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmModel.from_pretrained(model_name)

def get_embedding(sequence):
    inputs = tokenizer(sequence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state

subset_train_total['Embedding'] = subset_train_total['Sequence'].apply(get_embedding)

# print(subset_train_total[['EntryID', 'Sequence', 'Embedding']])
subset_train_total[['EntryID', 'Sequence', 'Embedding']]

/opt/anaconda3/envs/biotorch/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ESMTokenizer'. 
The class this function is called from is 'EsmTokenizer'.
Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm-1b and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight', 'esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/qc/w1hk3rk56nj4rcdqmlb7lrnw0000gn/T/ipykerne

,EntryID,Sequence,Embedding
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
1,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
2,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
3,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
4,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
5,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
6,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."


In [43]:
subset_train_total

,EntryID,Sequence,taxonomyID,term,Name,Namespace,Embedding
0,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0003674,molecular_function,molecular_function,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
1,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0005488,binding,molecular_function,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
2,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0005515,protein binding,molecular_function,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
3,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0006139,nucleobase-containing compound metabolic process,biological_process,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."
4,P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...,10249,GO:0006259,DNA metabolic process,biological_process,"[[[tensor(0.4123), tensor(0.0939), tensor(-0.0..."


In [44]:
# 提取GO术语并生成EL Embeddings
def extract_terms_and_generate_embeddings(graph, namespaces, embedding_dim, max_terms=5):
    terms = []
    embeddings = []
    count = 0
    for node, data in graph.nodes(data=True):
        if 'namespace' in data and data['namespace'] in namespaces:
            terms.append(node)
            embeddings.append(np.random.randn(embedding_dim))  # 示例中用随机向量表示
            count += 1
            if count >= max_terms:
                break
    return terms, np.array(embeddings)

# 假设EL Embedding的维度为256
embedding_dim = 256
graph = read_obo(par)
terms, embeddings = extract_terms_and_generate_embeddings(graph, {"molecular_function", "biological_process", "cellular_component"}, embedding_dim, max_terms=5)

# 将GO术语和嵌入表示存储在DataFrame中
terms_df = pd.DataFrame({'term': terms, 'EL_Embedding': list(embeddings)})

# 打印部分结果
terms_df

,term,EL_Embedding
0,GO:0000001,"[-0.38843499858703834, 0.5903889354963208, -0...."
1,GO:0000002,"[0.21704287287170954, -1.3415500185983502, -1...."
2,GO:0000003,"[-0.21563976788206762, -1.5438431535317647, 0...."
3,GO:0000006,"[1.3843893747114067, 0.643525358487707, -2.031..."
4,GO:0000007,"[-1.0187966626761569, -0.008660862465717607, 1..."


In [24]:
import numpy as np
import pandas as pd
import os
from Bio import SeqIO
from transformers import EsmTokenizer, EsmModel
import torch
import torch.nn as nn
import obonet

# 定义MLP模型
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [25]:
# 创建MLP模型实例
input_dim = 1280  # ESM2 Embedding的维度
output_dim = embedding_dim  # EL Embedding的维度
mlp = MLP(input_dim, output_dim)

# 投射蛋白质ESM Embedding
def project_embedding(esm_embedding):
    esm_embedding_tensor = torch.tensor(esm_embedding).float()
    projected_embedding = mlp(esm_embedding_tensor)
    return projected_embedding.detach().numpy()

# 对每个蛋白质序列的ESM Embedding进行投射
subset_train_total['Projected_Embedding'] = subset_train_total['Embedding'].apply(lambda x: project_embedding(x.mean(dim=1).squeeze().numpy()))

# 打印部分结果
print(subset_train_total.head())

# 合并EL Embeddings与蛋白质的Projected Embedding
final_df = pd.merge(subset_train_total, terms_df, on='term')

# 打印最终的结果
print(final_df.head())

/var/folders/qc/w1hk3rk56nj4rcdqmlb7lrnw0000gn/T/ipykernel_17746/2471869999.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_train_total['Projected_Embedding'] = subset_train_total['Embedding'].apply(lambda x: project_embedding(x.mean(dim=1).squeeze().numpy()))


  EntryID                                           Sequence  taxonomyID  \
0  P20536  MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...       10249   
1  P20536  MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...       10249   
2  P20536  MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...       10249   
3  P20536  MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...       10249   
4  P20536  MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...       10249   

         term                                              Name  \
0  GO:0003674                                molecular_function   
1  GO:0005488                                           binding   
2  GO:0005515                                   protein binding   
3  GO:0006139  nucleobase-containing compound metabolic process   
4  GO:0006259                             DNA metabolic process   

            Namespace                                          Embedding  \
0  molecular_function  [[[tensor(0.4123), tensor(0.0939), tensor

In [26]:
final_df

,EntryID,Sequence,taxonomyID,term,Name,Namespace,Embedding,Projected_Embedding,EL_Embedding


---

In [ ]:
model_name = "facebook/esm-1b"
tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmModel.from_pretrained(model_name)

def get_embedding(sequence):
    inputs = tokenizer(sequence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state

train_total['Embedding'] = train_total['Sequence'].apply(get_embedding)

train_total.head()

In [ ]:
# 提取GO术语并生成EL Embeddings
def extract_terms_and_generate_embeddings(graph, namespaces, embedding_dim):
    terms = []
    embeddings = []
    for node, data in graph.nodes(data=True):
        if 'namespace' in data and data['namespace'] in namespaces:
            terms.append(node)
            embeddings.append(np.random.randn(embedding_dim))  # 示例中用随机向量表示
    return terms, np.array(embeddings)

# 假设EL Embedding的维度为256
embedding_dim = 256
graph = read_obo(par)
terms, embeddings = extract_terms_and_generate_embeddings(graph, {"molecular_function", "biological_process", "cellular_component"}, embedding_dim)

# 将GO术语和嵌入表示存储在DataFrame中
terms_df = pd.DataFrame({'term': terms, 'EL_Embedding': list(embeddings)})

# 打印部分结果
print(terms_df.head())

In [ ]:
import numpy as np
import pandas as pd
import os
from Bio import SeqIO
from transformers import EsmTokenizer, EsmModel
import torch
import torch.nn as nn
import obonet

# 定义MLP模型
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建MLP模型实例
input_dim = 1280  # ESM2 Embedding的维度
output_dim = embedding_dim  # EL Embedding的维度
mlp = MLP(input_dim, output_dim)

# 投射蛋白质ESM Embedding
def project_embedding(esm_embedding):
    esm_embedding_tensor = torch.tensor(esm_embedding).float()
    projected_embedding = mlp(esm_embedding_tensor)
    return projected_embedding.detach().numpy()

# 对每个蛋白质序列的ESM Embedding进行投射
train_total['Projected_Embedding'] = train_total['Embedding'].apply(lambda x: project_embedding(x.mean(dim=1).squeeze().numpy()))

# 打印部分结果
print(train_total.head())

# 合并EL Embeddings与蛋白质的Projected Embedding
final_df = pd.merge(train_total, terms_df, on='term')

# 打印最终的结果
print(final_df.head())
